In [1]:
import pandas as pd

In [78]:
# 1. Load CSV files
orders = pd.read_csv("data\\olist_orders_dataset.csv")
items = pd.read_csv("data\\olist_order_items_dataset.csv")
payments = pd.read_csv("data\\olist_order_payments_dataset.csv")
customers = pd.read_csv("data\\olist_customers_dataset.csv")
products = pd.read_csv("data\\olist_products_dataset.csv")
sellers = pd.read_csv("data\\olist_sellers_dataset.csv")
reviews = pd.read_csv("data\\olist_order_reviews_dataset.csv")

# 2. Merge: Orders + Customers
master = pd.merge(orders, customers, on="customer_id", how="left")

# 3. Merge: Add order_items
master = pd.merge(master, items, on="order_id", how="left")

# 4. Merge: Add payments
master = pd.merge(master, payments, on="order_id", how="left")

# 5. Merge: Add products
master = pd.merge(master, products, on="product_id", how="left")

# 6. Merge: Add sellers
master = pd.merge(master, sellers, on="seller_id", how="left")

# 7. Merge: Add reviews
master = pd.merge(master, reviews, on="order_id", how="left")


In [79]:
#creating a new clean df with required cols
columns_needed = [
    'order_id','order_purchase_timestamp','price','freight_value',
    'payment_value','payment_type','customer_state','seller_state',
    'product_category_name','product_weight_g','product_length_cm',
    'product_height_cm','product_width_cm','review_score'
]

df_clean = master[columns_needed].copy()

In [80]:
# filled nulls in height,weight,length cols
df_clean['product_height_cm'] = df_clean['product_height_cm'].fillna(df_clean['product_height_cm'].mean())
df_clean['product_width_cm'] = df_clean['product_width_cm'].fillna(df_clean['product_width_cm'].mean())
df_clean['product_length_cm'] = df_clean['product_length_cm'].fillna(df_clean['product_length_cm'].mean())
# created col volume
df_clean['product_volume']=df_clean['product_height_cm']*df_clean['product_width_cm']*df_clean['product_length_cm']


In [81]:
#dropped height,weight,length cols
df_clean=df_clean.drop('product_height_cm',axis=1)
df_clean=df_clean.drop('product_width_cm',axis=1)
df_clean=df_clean.drop('product_length_cm',axis=1)

In [82]:
# filled nulls in weigth
df_clean['product_weight_g']=df_clean['product_weight_g'].fillna(df_clean['product_weight_g'].mean())

In [85]:
# cfilling nulls
df_clean['price']=df_clean['price'].fillna(df_clean['price'].mean())
df_clean['payment_value']=df_clean['payment_value'].fillna(df_clean['payment_value'].mean())
df_clean['freight_value']=df_clean['freight_value'].fillna(df_clean['freight_value'].mean())
df_clean['review_score']=df_clean['review_score'].fillna(df_clean['review_score'].mean())
df_clean['product_category_name']=df_clean['product_category_name'].fillna("Others")
df_clean['seller_state']=df_clean['seller_state'].fillna("Others")

In [87]:
state_map = {
    'AC': 'Acre',
    'AL': 'Alagoas',
    'AP': 'Amapá',
    'AM': 'Amazonas',
    'BA': 'Bahia',
    'CE': 'Ceará',
    'DF': 'Distrito Federal',
    'ES': 'Espírito Santo',
    'GO': 'Goiás',
    'MA': 'Maranhão',
    'MT': 'Mato Grosso',
    'MS': 'Mato Grosso do Sul',
    'MG': 'Minas Gerais',
    'PA': 'Pará',
    'PB': 'Paraíba',
    'PR': 'Paraná',
    'PE': 'Pernambuco',
    'PI': 'Piauí',
    'RJ': 'Rio de Janeiro',
    'RN': 'Rio Grande do Norte',
    'RS': 'Rio Grande do Sul',
    'RO': 'Rondônia',
    'RR': 'Roraima',
    'SC': 'Santa Catarina',
    'SP': 'São Paulo',
    'SE': 'Sergipe',
    'TO': 'Tocantins',
    'Others': 'Others'  #  filled values
}

In [88]:
#df_clean['seller_state']=df_clean['seller_state'].map(state_map)
df_clean['customer_state']=df_clean['customer_state'].map(state_map)

In [90]:
df_clean['seller_state']=df_clean['seller_state'].map(state_map)

In [92]:
df_clean['review_score']=df_clean['review_score'].astype(int)

In [ ]:
# created review review_category
def categorize(scr):
    if scr in [0,1,2]:
        return "Bad"
    elif scr==3:
        return "Average"
    else:
        return "Good"

df_clean['review_category']=df_clean['review_score'].apply(categorize)

,order_id,order_purchase_timestamp,price,freight_value,payment_value,payment_type,customer_state,seller_state,product_category_name,product_weight_g,review_score,product_volume,review_category
0,e481f51cbdc54678b7cc49136f2d6af7,2017-10-02 10:56:33,29.99,8.72,18.12,credit_card,São Paulo,São Paulo,utilidades_domesticas,500.0,4,1976.0,Good
1,e481f51cbdc54678b7cc49136f2d6af7,2017-10-02 10:56:33,29.99,8.72,2.00,voucher,São Paulo,São Paulo,utilidades_domesticas,500.0,4,1976.0,Good
2,e481f51cbdc54678b7cc49136f2d6af7,2017-10-02 10:56:33,29.99,8.72,18.59,voucher,São Paulo,São Paulo,utilidades_domesticas,500.0,4,1976.0,Good
3,53cdb2fc8bc7dce0b6741e2150273451,2018-07-24 20:41:37,118.70,22.76,141.46,boleto,Bahia,São Paulo,perfumaria,400.0,4,4693.0,Good
4,47770eb9100c2d0c44946d9cf07ec65d,2018-08-08 08:38:49,159.90,19.22,179.12,credit_card,Goiás,São Paulo,automotivo,420.0,5,9576.0,Good
...,...,...,...,...,...,...,...,...,...,...,...,...,...
119138,63943bddc261676b46f01ca7ac2f7bd8,2018-02-06 12:58:58,174.90,20.10,195.00,credit_card,São Paulo,São Paulo,bebes,4950.0,4,16000.0,Good
119139,83c1379a015df1e13d02aae0204711ab,2017-08-27 14:46:43,205.99,65.02,271.01,credit_card,Bahia,São Paulo,eletrodomesticos_2,13300.0,5,63360.0,Good
119140,11c177c8e97725db2631073c19f07b62,2018-01-08 21:28:27,179.99,40.59,441.16,credit_card,Rio de Janeiro,Minas Gerais,informatica_acessorios,6550.0,2,8000.0,Bad
119141,11c177c8e97725db2631073c19f07b62,2018-01-08 21:28:27,179.99,40.59,441.16,credit_card,Rio de Janeiro,Minas Gerais,informatica_acessorios,6550.0,2,8000.0,Bad
